In [ ]:
import numpy as np
import yaml

with open("../data/D_germline_ex.yaml", 'r') as stream:
    try:
        y = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Build a dictionary of the YAML keyed by state name.
state_d = {}
for s in y['states']:
    state_d[s['name']] = s

bases = list('ACGT')
insert_names = ['insert_left_'+x for x in bases]
dummy_names = ['dummy_D_'+str(i) for i in range(3)]

In [ ]:
def stack_rows(array_list):
    return np.concatenate([np.array([a]) for a in array_list])

def extract_normalized_transitions(d, names):
    v = np.array([d[n] for n in names])
    #v /= v.sum()
    return v

In [ ]:
# NTI transition and emission matrices.

in_landing = extract_normalized_transitions(state_d['init']['transitions'], insert_names)

T = stack_rows([
        extract_normalized_transitions(state_d[n]['transitions'], insert_names) 
        for n in insert_names])

E = stack_rows([
        np.array([state_d[n]['emissions']['probs'][b] for b in bases]) 
        for n in insert_names])

out_landing = stack_rows([
        extract_normalized_transitions(state_d[i_n]['transitions'], dummy_names) 
        for i_n in insert_names])

emission_indices = [0, 1, 0, 2, 3, 0, 1, 1, 1, 3, 2, 3, 3]
emission_indices = [2, 3, 0, 1, 1, 1]

in_landing, T, E, out_landing

In [ ]:
all_A = []
for t in range(len(emission_indices)):
    y_t = emission_indices[t]
    A = np.zeros((t+1, 4))
    if t > 0:
        A[:t, :] = all_A[t-1].dot(T) * E[y_t, :]
    A[t, :] = in_landing * E[y_t, :]
    all_A.append(A)

In [ ]:
all_A[-1].dot(out_landing)